In [ ]:
from IPython.display import display
import pandas as pd


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()


marseille_as_direction_flows = toflit_client.get_flows(
 customs_region = 'Marseille',
)
len(marseille_as_direction_flows)

In [ ]:
sources_as_direction = {}
for p in marseille_as_direction_flows:
    source = p["source_type"]
    action = p["export_import"]
    year = p["year"]
    if source not in sources_as_direction:
        sources_as_direction[source] = {}
    if year not in sources_as_direction[source]:
        sources_as_direction[source][year] = {}
    if action not in sources_as_direction[source][year]:
        sources_as_direction[source][year][action] = 0
    sources_as_direction[source][year][action] += 1

sources_as_direction_list = []
for source, yeargroup in sources_as_direction.items():
    for year, actionsgroup in yeargroup.items():
        for action, count in actionsgroup.items():
            sources_as_direction_list.append({
                "source": source,
                "year": year,
                "action": action,
                "count": count
            })

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par type, année et source pour Marseille comme direction des fermes",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 200,
    "data": {
        "values": sources_as_direction_list
    },
    "encoding": {
        "column": {
            "field": "source"
        },
        "x": {
            "field": "action",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "field": "count",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()


marseille_as_partner_flows = toflit_client.get_flows(
 partner_simplification = 'Marseille',
)
len(marseille_as_partner_flows)

In [ ]:
sources_as_partner = {}
for p in marseille_as_partner_flows:
    source = p["source_type"]
    action = p["export_import"]
    year = p["year"]
    if source not in sources_as_partner:
        sources_as_partner[source] = {}
    if year not in sources_as_partner[source]:
        sources_as_partner[source][year] = {}
    if action not in sources_as_partner[source][year]:
        sources_as_partner[source][year][action] = 0
    sources_as_partner[source][year][action] += 1

sources_as_partner_list = []
for source, yeargroup in sources_as_partner.items():
    for year, actionsgroup in yeargroup.items():
        for action, count in actionsgroup.items():
            sources_as_partner_list.append({
                "source": source,
                "year": year,
                "action": action,
                "count": count
            })

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Nombre de flux par type, année et source pour Marseille comme partenaire",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect"
    },
    "width": 300,
    "data": {
        "values": sources_as_partner_list
    },
    "encoding": {
        "column": {
            "field": "source"
        },
        "x": {
            "field": "action",
            "type": "nominal", 
            "title": "action",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "year",
            "type": "nominal",
            "title": "année"
        },
        "color": {
            "field": "count",
            "type": "quantitative",
            "title": "Nombre de flux"
        }
    }
})